# 메시지(Messages)

메시지는 LangChain에서 모델의 컨텍스트를 나타내는 기본 단위입니다. 모델의 입력과 출력을 나타내며, LLM과 상호 작용할 때 대화 상태를 나타내는 데 필요한 콘텐츠와 메타데이터를 모두 포함합니다.

메시지는 다음을 포함하는 객체입니다.

- **역할(Role)**: 메시지 유형을 식별 (예: `system`, `user`, `assistant`)
- **콘텐츠(Content)**: 메시지의 실제 콘텐츠를 나타냄 (텍스트, 이미지, 오디오, 문서 등)
- **메타데이터(Metadata)**: 응답 정보, 메시지 ID, 토큰 사용량 등의 선택적 필드

LangChain은 모델 제공자에 관계없이 일관된 동작을 보장하는 표준 메시지 유형을 제공합니다.

## 환경 설정

환경 변수를 설정합니다.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# 환경 변수 로드
load_dotenv(override=True)
# 추적을 위한 프로젝트 이름 설정
logging.langsmith("LangChain-V1-Tutorial")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangChain-V1-Tutorial


## 기본 사용법

메시지를 사용하는 가장 간단한 방법은 메시지 객체를 생성하고 모델을 호출할 때 전달하는 것입니다.

In [3]:
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 모델 초기화
model = init_chat_model("openai:gpt-4.1-mini")

# 메시지 객체 생성
system_msg = SystemMessage("당신은 친절한 Assistant입니다.")
human_msg = HumanMessage("안녕하세요. 반갑습니다.")

# 채팅 모델과 함께 사용
messages = [system_msg, human_msg]
response = model.invoke(messages)  # AIMessage 반환
response

AIMessage(content='안녕하세요! 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 27, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CeOaX8iMCjEQCKEUjYKwelwEYxjmh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--42a1b4c8-9eaa-4009-92f2-5bb3f06f1d63-0', usage_metadata={'input_tokens': 27, 'output_tokens': 14, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 텍스트 프롬프트

텍스트 프롬프트는 문자열입니다. 대화 기록을 유지할 필요가 없는 간단한 생성 작업에 이상적입니다.

In [4]:
# 단일 문자열로 간단한 요청
response = model.invoke("대한민국의 수도는?")
print(response.content)

대한민국의 수도는 서울특별시입니다.


## 메시지 프롬프트

메시지 객체 목록을 제공하여 모델에 메시지 목록을 전달할 수 있습니다.

참고: **메시지 프롬프트를 사용하는 경우**

- 다중 턴 대화를 관리하는 경우
- 멀티모달 콘텐츠(이미지, 오디오, 파일)를 작업하는 경우
- 시스템 지침을 포함하는 경우

In [5]:
from langchain.messages import SystemMessage, HumanMessage, AIMessage

# 메시지 객체를 사용한 대화
messages = [
    SystemMessage("당신은 친절한 어시스턴트입니다."),
    HumanMessage("대한민국의 수도는?"),
    AIMessage("대한민국의 수도는 서울입니다."),
    HumanMessage("영어로 작성해줘."),
]

response = model.invoke(messages)
print(response.content)

The capital of South Korea is Seoul.


### 딕셔너리 형식

아래와 같이 객체 형식이 아닌 딕셔너리 형식으로 메시지를 지정할 수도 있습니다.

In [6]:
# 딕셔너리 형식으로 메시지 지정
messages = [
    {"role": "system", "content": "당신은 친절한 어시스턴트입니다."},
    {"role": "user", "content": "대한민국의 수도는?"},
    {"role": "assistant", "content": "대한민국의 수도는 서울입니다."},
    {"role": "user", "content": "영어로 작성해줘."},
]

response = model.invoke(messages)
print(response.content)

The capital of South Korea is Seoul.


## 메시지 유형

LangChain은 다음과 같은 메시지 유형을 제공합니다.

- **시스템 메시지(System Message)**: 모델의 동작 방식을 알려주고 상호 작용을 위한 컨텍스트를 제공
- **사용자 메시지(Human Message)**: 사용자 입력 및 모델과의 상호 작용을 나타냄
- **AI 메시지(AI Message)**: 텍스트 콘텐츠, 도구 호출 및 메타데이터를 포함한 모델이 생성한 응답
- **도구 메시지(Tool Message)**: 도구 호출의 출력을 나타냄

### SystemMessage(시스템 메시지)

시스템 메시지는 모델의 동작을 준비하는 초기 지침 세트를 나타냅니다. 

시스템 메시지를 사용하여 톤을 설정하고, 모델의 역할을 정의하며, 응답에 대한 가이드라인을 수립할 수 있습니다.

In [7]:
from langchain.messages import SystemMessage, HumanMessage

# 기본 지침
system_msg = SystemMessage("You are a helpful python coding assistant.")

messages = [system_msg, HumanMessage("로또 번호 추첨 코드를 작성해줘.")]

response = model.invoke(messages)
print(response.content)

로또 번호 추첨 파이썬 코드를 작성해봤습니다. 일반적으로 로또는 1부터 45까지 숫자 중에서 6개를 중복 없이 무작위로 뽑는 방식입니다.

아래 코드를 참고하세요:

```python
import random

def generate_lotto_numbers():
    # 1부터 45까지 숫자 리스트에서 6개를 무작위로 선택
    numbers = random.sample(range(1, 46), 6)
    numbers.sort()  # 정렬해서 출력하면 보기 편함
    return numbers

if __name__ == "__main__":
    lotto_numbers = generate_lotto_numbers()
    print("이번 주 로또 번호:", lotto_numbers)
```

코드를 실행하면 매번 새로운 6개의 로또 번호가 출력됩니다. 필요하면 보너스 번호 추가 등으로 확장할 수도 있습니다!


### HumanMessage(사용자 메시지)

사용자 메시지는 사용자 입력 및 상호 작용을 나타냅니다. 텍스트, 이미지, 오디오, 파일 및 기타 멀티모달 콘텐츠를 포함할 수 있습니다.

In [8]:
from langchain.messages import HumanMessage

# 메시지 객체 사용
response = model.invoke([HumanMessage("What is machine learning?")])
print(response.content)

Machine learning is a branch of artificial intelligence (AI) that focuses on developing algorithms and statistical models that enable computers to learn from and make predictions or decisions based on data, without being explicitly programmed for every specific task. Instead of following fixed instructions, machine learning systems improve their performance over time by identifying patterns and relationships within data.

In essence, machine learning involves training a model on a dataset so that it can generalize and make accurate predictions or classifications on new, unseen data. Common types of machine learning include:

- **Supervised learning:** The model is trained on labeled data, where the input-output pairs are known.
- **Unsupervised learning:** The model identifies patterns or groupings in unlabeled data.
- **Reinforcement learning:** The model learns to make decisions by receiving rewards or penalties based on its actions.

Machine learning is widely used in applications s

#### 메시지 메타데이터

아래와 같이 metadata 를 추가하여 전달할 수 있습니다.

아래 예시는 `content` 외에도 `name` 과 `id` 를 추가하여 메시지에 메타데이터를 추가한 예시입니다.

In [9]:
# 메타데이터 추가
human_msg = HumanMessage(
    content="안녕하세요? 반가워요",
    name="teddy",  # 선택사항: 다른 사용자 식별
    id="abc123",  # 선택사항: 추적을 위한 고유 식별자
)

human_msg

HumanMessage(content='안녕하세요? 반가워요', additional_kwargs={}, response_metadata={}, name='teddy', id='abc123')

In [10]:
response = model.invoke([human_msg])
response

AIMessage(content='안녕하세요! 반가워요. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CeOe5ECs9cXdJpyOXITDe0ct8wLS5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--99327fc7-e9db-4848-bf98-7503ad8545d3-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### AIMessage(AI 메시지)

AI 메시지는 모델 호출의 출력을 나타냅니다. 

멀티모달 데이터, 도구 호출 및 나중에 액세스할 수 있는 제공자별 메타데이터를 포함할 수 있습니다.

In [12]:
# 모델 호출 시 AIMessage 반환
response = model.invoke("대한민국의 수도는?")
print(f"Type: {type(response)}")
print(f"Content: {response.content}")

Type: <class 'langchain_core.messages.ai.AIMessage'>
Content: 대한민국의 수도는 서울입니다.


## ToolMessage(도구 메시지)

도구 호출을 지원하는 모델의 경우 AI 메시지에 도구 호출이 포함될 수 있습니다. 

도구 메시지는 단일 도구 실행 결과를 모델에 다시 전달하는 데 사용됩니다.

![](assets/LangGraph-Messages-Tool-Message.png)

In [16]:
from langchain.messages import AIMessage, HumanMessage, ToolMessage

# 모델이 도구 호출을 수행한 후
ai_message = AIMessage(
    content="",
    tool_calls=[
        {"name": "get_weather", "args": {"location": "서울"}, "id": "call_123"}
    ],
)
ai_message

AIMessage(content='', additional_kwargs={}, response_metadata={}, tool_calls=[{'name': 'get_weather', 'args': {'location': '서울'}, 'id': 'call_123', 'type': 'tool_call'}])

`ai_message` 는 도구 호출을 포함하는 메시지입니다. `content` 는 빈 문자열이고, `tool_calls` 는 도구 호출 목록입니다.

Agent 가 도구 호출을 하게 되면 `content` 는 빈 문자열이고, `tool_calls` 는 도구 호출 목록을 반환합니다.

아래는 이어지는 `ToolMessage` 의 예시입니다.

도구를 호출한 `AIMessage` 의 `tool_call_id` 와 도구 호출 결과를 포함하는 `ToolMessage` 의 `tool_call_id` 가 일치해야 합니다.

In [18]:
# 도구 실행 및 결과 메시지 생성
weather_result = "날씨 맑음. 섭씨 10도"
tool_message = ToolMessage(
    content=weather_result, tool_call_id="call_123"  # 호출 ID와 일치해야 함
)
tool_message

ToolMessage(content='날씨 맑음. 섭씨 10도', tool_call_id='call_123')

사용자 질문(HumanMessage) -> 모델 도구 호출(AIMessage) -> 도구 호출 결과 메시지 생성(ToolMessage)

In [19]:
# 대화 계속
messages = [
    HumanMessage("서울의 날씨는 어때?"),
    ai_message,  # 모델의 도구 호출
    tool_message,  # 도구 실행 결과
]

response = model.invoke(messages)  # 모델이 결과 처리
print(response.content)

지금 서울은 맑고 기온은 섭씨 10도입니다. 도움이 필요하시면 말씀해 주세요!


## 메시지 콘텐츠

메시지의 콘텐츠는 모델로 전송되는 데이터의 페이로드로 생각할 수 있습니다. 

메시지에는 느슨하게 타입이 지정된 `content` 속성이 있어 문자열과 타입이 지정되지 않은 객체(예: 딕셔너리) 목록을 지원합니다.

In [24]:
from langchain.messages import HumanMessage
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-4.1-mini")

# 문자열 콘텐츠
human_message = HumanMessage("Hello, how are you?")
print("String content:", human_message.content)

# Provider 네이티브 형식 (예: OpenAI)
human_message = HumanMessage(
    content=[
        {"type": "text", "text": "다음은 어떤 내용인지 설명해줘"},
        {
            "type": "image_url",
            "image_url": {
                "url": "https://blog.langchain.com/content/images/2023/09/image.png"
            },
        },
    ]
)
print("Provider 네이티브 형식:", human_message.content)

response = model.invoke([human_message])
print(response.content)

String content: Hello, how are you?
Provider 네이티브 형식: [{'type': 'text', 'text': '다음은 어떤 내용인지 설명해줘'}, {'type': 'image_url', 'image_url': {'url': 'https://blog.langchain.com/content/images/2023/09/image.png'}}]
이 이미지는 데이터 처리 및 저장의 전반적인 흐름을 나타내고 있습니다. 

1. **Source (출처)**: 다양한 데이터 출처(예: 슬랙, 유튜브, 게임, 깃허브, 구글 드라이브, PDF, CSV, DOC, TXT, PPT, 트위터, 이메일 등)에서 데이터를 가져옵니다.

2. **Load (로드)**: 출처에서 데이터를 불러와 초기 형태로 저장합니다.

3. **Transform (변환)**: 데이터를 일정한 형식으로 변환하거나 전처리하는 과정입니다.

4. **Embed (임베드)**: 변환된 데이터를 벡터나 숫자 형태의 임베딩 표현으로 바꿉니다. 이는 머신러닝이나 검색에 활용할 수 있는 형태입니다.

5. **Store (저장)**: 임베딩된 데이터를 저장소에 저장합니다.

6. **Retrieve (검색/조회)**: 저장된 데이터를 필요할 때 조회하거나 검색하는 단계입니다.

요약하면, 이 그림은 다양한 원천 데이터로부터 데이터를 수집하고, 이를 정제 및 임베딩하여 저장한 후, 필요한 시점에 검색할 수 있게 하는 데이터 파이프라인 과정을 보여줍니다.


In [ ]:
# 표준 콘텐츠 블록 목록
human_message = HumanMessage(
    content_blocks=[
        {"type": "text", "text": "다음은 어떤 내용인지 설명해줘"},
        {
            "type": "image",
            "url": "https://blog.langchain.com/content/images/2023/09/image.png",
        },
    ]
)
print("표준 콘텐츠 블록:", human_message.content_blocks)

response = model.invoke([human_message])
print(response.content)

표준 콘텐츠 블록: [{'type': 'text', 'text': '다음은 어떤 내용인지 설명해줘'}, {'type': 'image', 'url': 'https://blog.langchain.com/content/images/2023/09/image.png'}]
이 이미지는 데이터 처리 파이프라인의 흐름을 설명하는 다이어그램입니다. 

1. **Source (데이터 소스)**: 다양한 형태와 출처(예: 슬랙, 유튜브, 게임, 깃허브, PDF, 문서, CSV, 이미지, 이메일, 웹페이지 등)에서 데이터를 수집하는 단계입니다.

2. **Load (로딩)**: 수집된 원본 데이터를 시스템에 로드하는 단계입니다.

3. **Transform (변환)**: 로드된 데이터를 필요한 형식으로 가공하거나 정제하는 단계입니다.

4. **Embed (임베딩)**: 텍스트나 데이터를 벡터 형태(숫자 배열)로 변환하는 단계입니다. 이는 머신러닝 모델이나 검색 시스템에서 활용할 수 있는 형태입니다.

5. **Store (저장)**: 임베딩된 데이터를 데이터베이스나 저장소에 저장하는 단계입니다.

6. **Retrieve (검색/조회)**: 저장된 데이터를 필요할 때 불러오는 단계입니다.

전체적으로 다양한 소스에서 데이터를 수집하여, 이를 정제 및 변환 후 벡터 임베딩하고, 벡터 데이터를 저장하고 나중에 검색할 수 있는 데이터 처리 및 관리의 전 과정을 시각화한 것입니다.
